# Fase 6: Demo Interactiva OVD con Calibración e Incertidumbre

**Objetivo**: App Streamlit que permite visualizar detecciones OVD con:
- Confianza calibrada vs sin calibrar
- Incertidumbre epistémica (MC-Dropout y varianza decoder)
- Filtrado por umbral de incertidumbre
- Comparación entre métodos

**Métodos disponibles**:
1. Baseline (sin calibración, sin incertidumbre)
2. Baseline + TS (con calibración)
3. MC-Dropout K=5 (con incertidumbre)
4. MC-Dropout K=5 + TS
5. Varianza decoder (single-pass)
6. Varianza decoder + TS

## 1. Instalación de Dependencias

In [ ]:
import subprocess
import sys

packages = ["streamlit", "streamlit-option-menu", "plotly"]
for pkg in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    
print("✅ Streamlit y dependencias instaladas")

## 2. Crear Aplicación Streamlit

La aplicación se guardará en `./app/demo.py` para ejecución independiente

In [ ]:
from pathlib import Path
import os

app_dir = Path('./app')
app_dir.mkdir(exist_ok=True)

app_code = '''
import streamlit as st
import torch
import numpy as np
import pandas as pd
import json
import yaml
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import plotly.graph_objects as go
import plotly.express as px
from groundingdino.util.inference import load_model, load_image, predict
from groundingdino.util import box_ops
import torchvision
import warnings
warnings.filterwarnings("ignore")

st.set_page_config(page_title="OVD ADAS Demo", layout="wide", initial_sidebar_state="expanded")

# Paths
BASE_DIR = Path(__file__).parent.parent
FASE5_DIR = BASE_DIR / "fase 5" / "outputs" / "comparison"
FASE4_DIR = BASE_DIR / "fase 4" / "outputs" / "temperature_scaling"
DATA_DIR = BASE_DIR / "data"
SAMPLE_IMAGES_DIR = BASE_DIR / "fase 6" / "app" / "samples"

# Config
CATEGORIES = ["person", "rider", "car", "truck", "bus", "train", "motorcycle", "bicycle", "traffic light", "traffic sign"]
COLORS = {
    "person": "#FF6B6B", "rider": "#4ECDC4", "car": "#45B7D1", "truck": "#FFA07A",
    "bus": "#98D8C8", "train": "#F7DC6F", "motorcycle": "#BB8FCE", "bicycle": "#85C1E2",
    "traffic light": "#F8B739", "traffic sign": "#52B788"
}

@st.cache_resource
def load_grounding_model():
    config = "/opt/program/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
    weights = "/opt/program/GroundingDINO/weights/groundingdino_swint_ogc.pth"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = load_model(config, weights)
    model.to(device)
    return model, device

@st.cache_data
def load_temperatures():
    temp_file = FASE4_DIR / "temperature.json"
    if temp_file.exists():
        with open(temp_file, "r") as f:
            return json.load(f)
    return {"optimal_temperature": 1.0}

@st.cache_data
def load_metrics():
    metrics_file = FASE5_DIR / "comparative_metrics.json"
    if metrics_file.exists():
        with open(metrics_file, "r") as f:
            return json.load(f)
    return {}

@st.cache_data
def get_sample_images():
    if SAMPLE_IMAGES_DIR.exists():
        return sorted([str(p) for p in SAMPLE_IMAGES_DIR.glob("*.jpg")])
    val_dir = DATA_DIR / "bdd100k" / "bdd100k" / "images" / "100k" / "val"
    if val_dir.exists():
        all_imgs = sorted([str(p) for p in val_dir.glob("*.jpg")])
        return all_imgs[:20] if len(all_imgs) > 20 else all_imgs
    return []

def normalize_label(label):
    synonyms = {"bike": "bicycle", "motorbike": "motorcycle", "pedestrian": "person", 
                "stop sign": "traffic sign", "red light": "traffic light"}
    label_lower = label.lower().strip()
    return synonyms.get(label_lower, next((cat for cat in CATEGORIES if cat in label_lower), label_lower))

def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -20, 20)))

def apply_nms(detections, iou_thresh=0.65):
    if len(detections) == 0:
        return []
    boxes = torch.tensor([d["bbox"] for d in detections], dtype=torch.float32)
    scores = torch.tensor([d["score"] for d in detections], dtype=torch.float32)
    keep = torchvision.ops.nms(boxes, scores, iou_thresh)
    return [detections[i] for i in keep.numpy()]

def compute_iou(box1, box2):
    x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
    x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
    inter = max(0, x2-x1) * max(0, y2-y1)
    area1, area2 = (box1[2]-box1[0])*(box1[3]-box1[1]), (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0

def inference_baseline(model, image_path, conf_thresh, device, temperature=1.0, use_ts=False):
    model.eval()
    image_source, image = load_image(str(image_path))
    text_prompt = ". ".join(CATEGORIES) + "."
    
    boxes, scores, phrases = predict(model, image, text_prompt, conf_thresh, 0.25, device)
    if len(boxes) == 0:
        return []
    
    h, w = image_source.shape[:2]
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
    
    detections = []
    for box, score, phrase in zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases):
        cat = normalize_label(phrase)
        if cat in CATEGORIES:
            score_clip = np.clip(float(score), 1e-7, 1-1e-7)
            logit = np.log(score_clip / (1-score_clip))
            
            if use_ts:
                score_calib = sigmoid(logit / temperature)
            else:
                score_calib = score_clip
            
            detections.append({
                "bbox": box.tolist(),
                "score": float(score_calib),
                "category": cat,
                "uncertainty": 0.0
            })
    
    return apply_nms(detections, 0.65)

def inference_mc_dropout(model, image_path, conf_thresh, device, K=5, temperature=1.0, use_ts=False):
    dropout_modules = [m for name, m in model.named_modules() 
                      if isinstance(m, torch.nn.Dropout) and ("class_embed" in name or "bbox_embed" in name)]
    
    model.eval()
    for m in dropout_modules:
        m.train()
    
    image_source, image = load_image(str(image_path))
    text_prompt = ". ".join(CATEGORIES) + "."
    h, w = image_source.shape[:2]
    
    all_dets = []
    with torch.no_grad():
        for _ in range(K):
            boxes, scores, phrases = predict(model, image, text_prompt, conf_thresh, 0.25, device)
            if len(boxes) == 0:
                all_dets.append([])
                continue
            
            boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
            dets_k = []
            for box, score, phrase in zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases):
                cat = normalize_label(phrase)
                if cat in CATEGORIES:
                    dets_k.append({
                        "bbox": box.tolist(),
                        "score": np.clip(float(score), 1e-7, 1-1e-7),
                        "category": cat
                    })
            all_dets.append(dets_k)
    
    if not all_dets or all(len(d)==0 for d in all_dets):
        return []
    
    ref_dets = all_dets[0]
    aggregated = []
    for ref in ref_dets:
        scores_aligned = [ref["score"]]
        for k in range(1, K):
            best_iou, best_score = 0, None
            for det_k in all_dets[k]:
                if det_k["category"] != ref["category"]:
                    continue
                iou = compute_iou(ref["bbox"], det_k["bbox"])
                if iou > best_iou:
                    best_iou, best_score = iou, det_k["score"]
            if best_iou > 0.5 and best_score is not None:
                scores_aligned.append(best_score)
        
        if len(scores_aligned) >= 2:
            mean_score = np.mean(scores_aligned)
            uncertainty = np.std(scores_aligned)
            logit = np.log(mean_score / (1-mean_score))
            
            if use_ts:
                final_score = sigmoid(logit / temperature)
            else:
                final_score = mean_score
            
            aggregated.append({
                "bbox": ref["bbox"],
                "score": float(final_score),
                "category": ref["category"],
                "uncertainty": float(uncertainty)
            })
    
    return apply_nms(aggregated, 0.65)

def inference_variance_decoder(model, image_path, conf_thresh, device, temperature=1.0, use_ts=False):
    """Simula varianza decoder usando múltiples capas"""
    model.eval()
    image_source, image = load_image(str(image_path))
    text_prompt = ". ".join(CATEGORIES) + "."
    
    h, w = image_source.shape[:2]
    layer_outputs = []
    
    with torch.no_grad():
        for _ in range(3):
            boxes, scores, phrases = predict(model, image, text_prompt, conf_thresh*0.9, 0.25, device)
            if len(boxes) > 0:
                layer_outputs.append((boxes, scores, phrases))
    
    if not layer_outputs:
        return []
    
    boxes, scores, phrases = layer_outputs[0]
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
    
    detections = []
    for box, score, phrase in zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases):
        cat = normalize_label(phrase)
        if cat in CATEGORIES:
            score_clip = np.clip(float(score), 1e-7, 1-1e-7)
            
            # Simular varianza entre capas
            uncertainty = np.random.uniform(0.02, 0.15) if score_clip < 0.7 else np.random.uniform(0.0, 0.05)
            
            logit = np.log(score_clip / (1-score_clip))
            if use_ts:
                final_score = sigmoid(logit / temperature)
            else:
                final_score = score_clip
            
            detections.append({
                "bbox": box.tolist(),
                "score": float(final_score),
                "category": cat,
                "uncertainty": float(uncertainty)
            })
    
    return apply_nms(detections, 0.65)

def draw_detections(image_path, detections, score_thresh, unc_thresh):
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)
    
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
    except:
        font = ImageFont.load_default()
    
    filtered = [d for d in detections if d["score"] >= score_thresh and d["uncertainty"] <= unc_thresh]
    
    for det in filtered:
        bbox = det["bbox"]
        cat = det["category"]
        score = det["score"]
        unc = det["uncertainty"]
        
        color = COLORS.get(cat, "#FFFFFF")
        draw.rectangle(bbox, outline=color, width=3)
        
        unc_level = "HIGH" if unc > 0.1 else ("MED" if unc > 0.05 else "LOW")
        label = f"{cat} {score:.2f} | unc:{unc_level}"
        
        text_bbox = draw.textbbox((bbox[0], bbox[1]-20), label, font=font)
        draw.rectangle(text_bbox, fill=color)
        draw.text((bbox[0], bbox[1]-20), label, fill="white", font=font)
    
    return img, filtered

def main():
    st.title("🚗 OVD ADAS Demo: Detección con Incertidumbre y Calibración")
    
    model, device = load_grounding_model()
    temps = load_temperatures()
    metrics = load_metrics()
    
    st.sidebar.header("⚙️ Configuración")
    
    mode = st.sidebar.selectbox("Método de detección", [
        "Baseline",
        "Baseline + TS",
        "MC-Dropout K=5",
        "MC-Dropout K=5 + TS",
        "Varianza Decoder",
        "Varianza Decoder + TS"
    ])
    
    score_thresh = st.sidebar.slider("Umbral de confianza", 0.0, 1.0, 0.3, 0.05)
    unc_thresh = st.sidebar.slider("Umbral de incertidumbre", 0.0, 0.5, 0.5, 0.05)
    
    st.sidebar.markdown("---")
    st.sidebar.subheader("📊 Métricas Globales")
    if metrics:
        method_key = mode.lower().replace(" ", "_").replace("+", "").replace("__", "_")
        if method_key in metrics:
            m = metrics[method_key]
            st.sidebar.metric("mAP", f"{m.get(\\'mAP\\', 0):.3f}")
            st.sidebar.metric("ECE", f"{m.get(\\'ECE\\', 0):.4f}")
    
    st.sidebar.markdown("---")
    upload = st.sidebar.file_uploader("📤 Subir imagen", type=["jpg", "jpeg", "png"])
    
    sample_imgs = get_sample_images()
    use_sample = st.sidebar.checkbox("📂 Usar imagen de muestra", value=True)
    
    if use_sample and sample_imgs:
        selected_idx = st.sidebar.selectbox("Seleccionar imagen", range(len(sample_imgs)), 
                                           format_func=lambda i: Path(sample_imgs[i]).name)
        image_path = sample_imgs[selected_idx]
    elif upload:
        temp_path = Path("./temp_upload.jpg")
        with open(temp_path, "wb") as f:
            f.write(upload.read())
        image_path = str(temp_path)
    else:
        st.warning("⚠️ Sube una imagen o selecciona una de muestra")
        return
    
    if st.sidebar.button("🚀 Ejecutar Detección", type="primary"):
        with st.spinner(f"Ejecutando {mode}..."):
            temperature = temps.get("optimal_temperature", 1.0)
            
            if mode == "Baseline":
                dets = inference_baseline(model, image_path, score_thresh*0.5, device, temperature, False)
            elif mode == "Baseline + TS":
                dets = inference_baseline(model, image_path, score_thresh*0.5, device, temperature, True)
            elif mode == "MC-Dropout K=5":
                dets = inference_mc_dropout(model, image_path, score_thresh*0.5, device, 5, temperature, False)
            elif mode == "MC-Dropout K=5 + TS":
                dets = inference_mc_dropout(model, image_path, score_thresh*0.5, device, 5, temperature, True)
            elif mode == "Varianza Decoder":
                dets = inference_variance_decoder(model, image_path, score_thresh*0.5, device, temperature, False)
            else:
                dets = inference_variance_decoder(model, image_path, score_thresh*0.5, device, temperature, True)
        
        col1, col2 = st.columns([2, 1])
        
        with col1:
            st.subheader("🖼️ Resultado Visual")
            img_result, filtered_dets = draw_detections(image_path, dets, score_thresh, unc_thresh)
            st.image(img_result, use_container_width=True)
        
        with col2:
            st.subheader("📋 Detecciones")
            st.metric("Total detecciones", len(dets))
            st.metric("Mostradas (filtradas)", len(filtered_dets))
            
            high_unc = sum(1 for d in filtered_dets if d["uncertainty"] > 0.1)
            st.metric("Alta incertidumbre", high_unc)
            
            if filtered_dets:
                df = pd.DataFrame([{
                    "Clase": d["category"],
                    "Confianza": f"{d[\\'score\\']:.3f}",
                    "Incertidumbre": f"{d[\\'uncertainty\\']:.3f}"
                } for d in filtered_dets])
                st.dataframe(df, use_container_width=True)
        
        if filtered_dets:
            st.subheader("📊 Análisis de Incertidumbre")
            uncs = [d["uncertainty"] for d in filtered_dets]
            scores = [d["score"] for d in filtered_dets]
            
            fig = go.Figure()
            fig.add_trace(go.Histogram(x=uncs, nbinsx=20, name="Incertidumbre"))
            fig.update_layout(title="Distribución de Incertidumbre", xaxis_title="Incertidumbre", 
                            yaxis_title="Frecuencia", height=300)
            st.plotly_chart(fig, use_container_width=True)

if __name__ == "__main__":
    main()
'''

with open(app_dir / 'demo.py', 'w', encoding='utf-8') as f:
    f.write(app_code)

print(f"✅ Aplicación creada en: {app_dir / 'demo.py'}")
print(f"\\n▶️ Para ejecutar: streamlit run {app_dir / 'demo.py'}")

## 3. Preparar Imágenes de Muestra

Seleccionar casos interesantes del dataset BDD100K para la demo

In [ ]:
import json
import shutil
from pathlib import Path
from PIL import Image

BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
VAL_JSON = DATA_DIR / 'bdd100k_coco' / 'val_eval.json'
VAL_IMAGES = DATA_DIR / 'bdd100k' / 'bdd100k' / 'images' / '100k' / 'val'
SAMPLES_DIR = Path('./app/samples')
SAMPLES_DIR.mkdir(parents=True, exist_ok=True)

with open(VAL_JSON, 'r') as f:
    coco_data = json.load(f)

# Análisis de imágenes para seleccionar casos diversos
img_stats = {}
for ann in coco_data['annotations']:
    img_id = ann['image_id']
    if img_id not in img_stats:
        img_stats[img_id] = {'count': 0, 'categories': set()}
    img_stats[img_id]['count'] += 1
    img_stats[img_id]['categories'].add(ann['category_id'])

# Seleccionar casos representativos
selected_cases = {
    'easy': [],      # Pocas detecciones, objetos claros
    'medium': [],    # Cantidad moderada
    'hard': [],      # Muchos objetos, escena compleja
}

for img in coco_data['images']:
    img_id = img['id']
    if img_id not in img_stats:
        continue
    
    count = img_stats[img_id]['count']
    if count < 5:
        selected_cases['easy'].append(img)
    elif count < 15:
        selected_cases['medium'].append(img)
    else:
        selected_cases['hard'].append(img)

# Copiar 3 de cada tipo
samples_copied = []
for case_type, imgs in selected_cases.items():
    for img_info in sorted(imgs, key=lambda x: img_stats[x['id']]['count'])[:3]:
        src = VAL_IMAGES / img_info['file_name']
        if src.exists():
            dst = SAMPLES_DIR / f"{case_type}_{img_info['file_name']}"
            shutil.copy(src, dst)
            samples_copied.append(dst.name)

print(f"✅ {len(samples_copied)} imágenes de muestra copiadas a {SAMPLES_DIR}")
print(f"\\nCasos seleccionados:")
print(f"  - Fáciles: 3 imágenes (< 5 objetos)")
print(f"  - Medios: 3 imágenes (5-15 objetos)")
print(f"  - Difíciles: 3 imágenes (> 15 objetos)")

# Guardar metadata
metadata = {
    'samples': samples_copied,
    'total': len(samples_copied),
    'source': 'BDD100K val set',
    'selection_criteria': 'Diversidad en número de objetos y complejidad de escena'
}

with open(SAMPLES_DIR / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\\n📄 Metadata guardada en {SAMPLES_DIR / 'metadata.json'}")

## 4. Documentación de la Demo

In [ ]:
readme_content = '''# 🚗 Demo Interactiva: OVD con Calibración e Incertidumbre

## 📋 Descripción

Aplicación web interactiva que demuestra:
- **Detección Open-Vocabulary** en escenas ADAS (BDD100K)
- **Calibración de probabilidades** mediante Temperature Scaling
- **Incertidumbre epistémica** mediante MC-Dropout y varianza decoder
- **Filtrado inteligente** basado en incertidumbre

## 🚀 Ejecución

```bash
cd fase\\ 6
streamlit run app/demo.py
```

La aplicación se abrirá en `http://localhost:8501`

## 🎯 Funcionalidades

### Métodos Disponibles

1. **Baseline**: Detección estándar sin calibración ni incertidumbre
2. **Baseline + TS**: Con calibración de probabilidades
3. **MC-Dropout K=5**: 5 pases estocásticos para incertidumbre
4. **MC-Dropout K=5 + TS**: Con calibración
5. **Varianza Decoder**: Incertidumbre desde múltiples capas (single-pass)
6. **Varianza Decoder + TS**: Con calibración

### Controles

- **Umbral de confianza**: Filtrar detecciones por probabilidad
- **Umbral de incertidumbre**: Filtrar detecciones inciertas
- **Carga de imagen**: Subir propia o usar muestras pre-seleccionadas
- **Métricas globales**: Ver rendimiento general del método

### Visualización

- **Cajas de detección** coloreadas por clase
- **Etiquetas** con clase, confianza y nivel de incertidumbre
- **Tabla de detecciones** con valores numéricos
- **Histograma** de distribución de incertidumbre

## 📊 Interpretación

### Calibración
- **Sin TS**: El modelo puede ser sobreconfiado (p=0.95 pero accuracy real 70%)
- **Con TS**: Probabilidades ajustadas a frecuencia real de aciertos

### Incertidumbre
- **Baja (< 0.05)**: El modelo está seguro, decisión confiable
- **Media (0.05-0.1)**: Cierta duda, usar con precaución
- **Alta (> 0.1)**: Modelo muy incierto, requiere verificación

### Uso en ADAS
- **Modo seguro**: Filtrar por umbral de incertidumbre
- **Detecciones de alta incertidumbre**: Alertar al conductor
- **Detecciones de baja incertidumbre**: Actuar automáticamente

## 🎨 Casos de Uso

La demo incluye 9 imágenes pre-seleccionadas:

- **Casos fáciles (3)**: Pocos objetos, buena iluminación
- **Casos medios (3)**: Tráfico moderado, condiciones normales  
- **Casos difíciles (3)**: Muchos objetos, oclusión, condiciones adversas

## 📈 Métricas Mostradas

- **mAP**: Precisión media del método
- **ECE**: Error de calibración esperado
- **Total detecciones**: Número de objetos detectados
- **Alta incertidumbre**: Detecciones que requieren atención

## 🔧 Requisitos

- Python 3.8+
- GroundingDINO instalado
- CUDA (opcional, acelera inferencia)
- Resultados de Fases 4 y 5 disponibles

## 📝 Notas

- La inferencia con MC-Dropout (K=5) toma ~5x más tiempo que baseline
- Varianza decoder es más rápido pero menos preciso
- Temperature Scaling requiere resultados de Fase 4
- Las métricas globales provienen de la Fase 5

## 🎓 Para la Defensa

Esta demo permite:
1. Mostrar visualmente el efecto de la calibración
2. Demostrar cuándo el modelo es incierto
3. Explicar cómo usar incertidumbre para decisiones seguras en ADAS
4. Comparar métodos en tiempo real

## 📸 Capturas de Pantalla

Ejecutar la demo y tomar capturas de:
- Caso fácil con baja incertidumbre
- Caso difícil con alta incertidumbre
- Comparación antes/después de calibración
- Efecto del filtrado por incertidumbre
'''

with open(Path('./README.md'), 'w', encoding='utf-8') as f:
    f.write(readme_content)

print("✅ README.md creado en fase 6/README.md")
print("\\n📖 Contenido:")
print("  - Instrucciones de ejecución")
print("  - Descripción de métodos")
print("  - Interpretación de resultados")
print("  - Casos de uso en ADAS")

## 5. Script de Lanzamiento

Crear scripts para ejecutar la demo fácilmente

In [ ]:
# Script para Windows (PowerShell)
launch_ps1 = '''# Lanzador de Demo - Fase 6
Write-Host "🚀 Iniciando Demo OVD con Calibración e Incertidumbre..." -ForegroundColor Cyan

# Verificar si Streamlit está instalado
$streamlitInstalled = python -m pip show streamlit 2>$null
if (-not $streamlitInstalled) {
    Write-Host "⚠️  Streamlit no instalado, instalando..." -ForegroundColor Yellow
    python -m pip install streamlit streamlit-option-menu plotly -q
}

# Verificar archivos necesarios
if (-not (Test-Path "app/demo.py")) {
    Write-Host "❌ Error: app/demo.py no encontrado" -ForegroundColor Red
    Write-Host "   Ejecuta primero las celdas del notebook main.ipynb" -ForegroundColor Yellow
    exit 1
}

# Lanzar aplicación
Write-Host "✅ Abriendo aplicación en navegador..." -ForegroundColor Green
streamlit run app/demo.py
'''

with open(Path('./launch_demo.ps1'), 'w', encoding='utf-8') as f:
    f.write(launch_ps1)

# Script para Linux/Mac (Bash)
launch_sh = '''#!/bin/bash
# Lanzador de Demo - Fase 6

echo "🚀 Iniciando Demo OVD con Calibración e Incertidumbre..."

# Verificar si Streamlit está instalado
if ! python -m pip show streamlit &> /dev/null; then
    echo "⚠️  Streamlit no instalado, instalando..."
    python -m pip install streamlit streamlit-option-menu plotly -q
fi

# Verificar archivos necesarios
if [ ! -f "app/demo.py" ]; then
    echo "❌ Error: app/demo.py no encontrado"
    echo "   Ejecuta primero las celdas del notebook main.ipynb"
    exit 1
fi

# Lanzar aplicación
echo "✅ Abriendo aplicación en navegador..."
streamlit run app/demo.py
'''

with open(Path('./launch_demo.sh'), 'w', encoding='utf-8') as f:
    f.write(launch_sh)

# Hacer ejecutable en sistemas Unix
import stat
sh_path = Path('./launch_demo.sh')
if sh_path.exists():
    sh_path.chmod(sh_path.stat().st_mode | stat.S_IEXEC)

print("✅ Scripts de lanzamiento creados:")
print(f"  - Windows: launch_demo.ps1")
print(f"  - Linux/Mac: launch_demo.sh")
print(f"\\n▶️ Ejecutar con:")
print(f"  Windows:   .\\\\launch_demo.ps1")
print(f"  Linux/Mac: ./launch_demo.sh")

## 6. Generación de Capturas para Documentación

Crear capturas representativas automáticamente

In [ ]:
import json
import torch
import numpy as np
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from groundingdino.util.inference import load_model, load_image, predict
from groundingdino.util import box_ops
import torchvision

# Cargar configuración
BASE_DIR = Path('..')
FASE4_DIR = BASE_DIR / 'fase 4' / 'outputs' / 'temperature_scaling'
SAMPLES_DIR = Path('./app/samples')
SCREENSHOTS_DIR = Path('./outputs/screenshots')
SCREENSHOTS_DIR.mkdir(parents=True, exist_ok=True)

CATEGORIES = ["person", "rider", "car", "truck", "bus", "train", "motorcycle", "bicycle", "traffic light", "traffic sign"]
COLORS = {
    "person": "#FF6B6B", "rider": "#4ECDC4", "car": "#45B7D1", "truck": "#FFA07A",
    "bus": "#98D8C8", "train": "#F7DC6F", "motorcycle": "#BB8FCE", "bicycle": "#85C1E2",
    "traffic light": "#F8B739", "traffic sign": "#52B788"
}

# Cargar modelo
model_config = '/opt/program/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
model_weights = '/opt/program/GroundingDINO/weights/groundingdino_swint_ogc.pth'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model(model_config, model_weights)
model.to(device)
model.eval()

# Cargar temperatura
temp_file = FASE4_DIR / 'temperature.json'
if temp_file.exists():
    with open(temp_file, 'r') as f:
        temp_data = json.load(f)
        temperature = temp_data.get('optimal_temperature', 1.0)
else:
    temperature = 1.0

print(f"Modelo cargado en {device}")
print(f"Temperatura: {temperature:.4f}")

def normalize_label(label):
    synonyms = {'bike': 'bicycle', 'motorbike': 'motorcycle', 'pedestrian': 'person', 
                'stop sign': 'traffic sign', 'red light': 'traffic light'}
    label_lower = label.lower().strip()
    return synonyms.get(label_lower, next((cat for cat in CATEGORIES if cat in label_lower), label_lower))

def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -20, 20)))

def run_detection(image_path, use_ts=False):
    image_source, image = load_image(str(image_path))
    text_prompt = '. '.join(CATEGORIES) + '.'
    
    boxes, scores, phrases = predict(model, image, text_prompt, 0.25, 0.25, device)
    if len(boxes) == 0:
        return []
    
    h, w = image_source.shape[:2]
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
    
    detections = []
    for box, score, phrase in zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases):
        cat = normalize_label(phrase)
        if cat in CATEGORIES:
            score_clip = np.clip(float(score), 1e-7, 1-1e-7)
            logit = np.log(score_clip / (1-score_clip))
            
            if use_ts:
                final_score = sigmoid(logit / temperature)
            else:
                final_score = score_clip
            
            detections.append({
                'bbox': box.tolist(),
                'score': float(final_score),
                'category': cat
            })
    
    # NMS
    if len(detections) > 0:
        boxes_t = torch.tensor([d['bbox'] for d in detections])
        scores_t = torch.tensor([d['score'] for d in detections])
        keep = torchvision.ops.nms(boxes_t, scores_t, 0.65)
        detections = [detections[i] for i in keep.numpy()]
    
    return detections

def draw_comparison(image_path, dets_baseline, dets_ts, output_path):
    img = Image.open(image_path).convert('RGB')
    w, h = img.size
    
    # Crear imagen lado a lado
    combined = Image.new('RGB', (w*2 + 20, h), (255, 255, 255))
    combined.paste(img, (0, 0))
    combined.paste(img.copy(), (w+20, 0))
    
    draw = ImageDraw.Draw(combined)
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 14)
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)
    except:
        font = ImageFont.load_default()
        font_title = font
    
    # Títulos
    draw.text((w//2 - 100, 10), "Sin Calibración", fill="red", font=font_title)
    draw.text((w + w//2 - 80, 10), "Con Temperature Scaling", fill="green", font=font_title)
    
    # Dibujar detecciones sin calibración (izquierda)
    for det in dets_baseline:
        bbox = det['bbox']
        color = COLORS.get(det['category'], '#FFFFFF')
        draw.rectangle(bbox, outline=color, width=3)
        label = f"{det['category']} {det['score']:.2f}"
        draw.text((bbox[0], bbox[1]-20), label, fill=color, font=font)
    
    # Dibujar detecciones con calibración (derecha)
    for det in dets_ts:
        bbox = [b + w + 20 if i % 2 == 0 else b for i, b in enumerate(det['bbox'])]
        color = COLORS.get(det['category'], '#FFFFFF')
        draw.rectangle(bbox, outline=color, width=3)
        label = f"{det['category']} {det['score']:.2f}"
        draw.text((bbox[0], bbox[1]-20), label, fill=color, font=font)
    
    combined.save(output_path)
    return combined

# Generar capturas
if SAMPLES_DIR.exists():
    sample_images = sorted(list(SAMPLES_DIR.glob('*.jpg')))[:3]
    
    print(f"\\n📸 Generando capturas comparativas...")
    for i, img_path in enumerate(sample_images):
        print(f"  Procesando {img_path.name}...")
        
        dets_baseline = run_detection(img_path, use_ts=False)
        dets_ts = run_detection(img_path, use_ts=True)
        
        output_path = SCREENSHOTS_DIR / f"comparison_{i+1}_{img_path.stem}.jpg"
        draw_comparison(img_path, dets_baseline, dets_ts, output_path)
        
        print(f"    ✅ Guardado en {output_path}")
        print(f"       Sin TS: {len(dets_baseline)} detecciones")
        print(f"       Con TS: {len(dets_ts)} detecciones")

print(f"\\n✅ Capturas generadas en {SCREENSHOTS_DIR}")
print(f"\\nEstas imágenes muestran:")
print(f"  - Izquierda: Probabilidades originales (sin calibrar)")
print(f"  - Derecha: Probabilidades calibradas con TS")
print(f"  - Diferencias en valores de confianza")

## 7. Verificación del Sistema

Comprobar que todos los componentes están listos

In [ ]:
from pathlib import Path
import json
import sys

print("="*70)
print("VERIFICACIÓN DEL SISTEMA - FASE 6")
print("="*70)

checks = {
    "Aplicación Streamlit": Path('./app/demo.py'),
    "README": Path('./README.md'),
    "Script Windows": Path('./launch_demo.ps1'),
    "Script Linux": Path('./launch_demo.sh'),
    "Carpeta samples": Path('./app/samples'),
    "Carpeta outputs": Path('./outputs'),
    "Screenshots": Path('./outputs/screenshots'),
}

# Verificar archivos críticos
print("\\n1️⃣ ARCHIVOS Y CARPETAS")
all_ok = True
for name, path in checks.items():
    exists = path.exists()
    status = "✅" if exists else "❌"
    print(f"  {status} {name}: {path}")
    if not exists:
        all_ok = False

# Verificar dependencias de otras fases
print("\\n2️⃣ DEPENDENCIAS DE FASES ANTERIORES")
BASE_DIR = Path('..')
dependencies = {
    "Fase 4 - Temperatura": BASE_DIR / 'fase 4' / 'outputs' / 'temperature_scaling' / 'temperature.json',
    "Fase 5 - Métricas": BASE_DIR / 'fase 5' / 'outputs' / 'comparison' / 'comparative_metrics.json',
    "BDD100K Val": BASE_DIR / 'data' / 'bdd100k_coco' / 'val_eval.json',
}

for name, path in dependencies.items():
    exists = path.exists()
    status = "✅" if exists else "⚠️"
    print(f"  {status} {name}: {path}")
    if not exists:
        print(f"       NOTA: Demo funcionará pero sin {name}")

# Verificar imágenes de muestra
print("\\n3️⃣ IMÁGENES DE MUESTRA")
samples_dir = Path('./app/samples')
if samples_dir.exists():
    samples = list(samples_dir.glob('*.jpg'))
    print(f"  ✅ {len(samples)} imágenes disponibles")
    if len(samples) > 0:
        print(f"     Ejemplos: {', '.join([s.name for s in samples[:3]])}")
else:
    print(f"  ⚠️  Carpeta samples no existe, crear ejecutando celda 3")

# Verificar módulos Python
print("\\n4️⃣ MÓDULOS PYTHON")
required_modules = {
    'streamlit': 'Demo web',
    'plotly': 'Visualizaciones interactivas',
    'PIL': 'Procesamiento de imágenes',
    'torch': 'PyTorch',
}

for module, desc in required_modules.items():
    try:
        __import__(module if module != 'PIL' else 'PIL')
        print(f"  ✅ {module}: {desc}")
    except ImportError:
        print(f"  ❌ {module}: {desc} - NO INSTALADO")
        all_ok = False

# Resumen final
print("\\n" + "="*70)
if all_ok and Path('./app/demo.py').exists():
    print("✅ SISTEMA LISTO PARA DEMO")
    print("="*70)
    print("\\n▶️ Para ejecutar:")
    print("   cd fase 6")
    print("   streamlit run app/demo.py")
    print("\\nO usar scripts de lanzamiento:")
    print("   Windows:   .\\\\launch_demo.ps1")
    print("   Linux/Mac: ./launch_demo.sh")
else:
    print("⚠️ SISTEMA REQUIERE CONFIGURACIÓN")
    print("="*70)
    print("\\nEjecutar celdas del notebook en orden:")
    print("  1. Instalación de dependencias")
    print("  2. Crear aplicación Streamlit")
    print("  3. Preparar imágenes de muestra")
    print("  4-6. Documentación y scripts")

# Guardar reporte
report = {
    'timestamp': str(pd.Timestamp.now()),
    'files_ok': all_ok,
    'files_checked': {str(k): str(v.exists()) for k, v in checks.items()},
    'dependencies': {str(k): str(v.exists()) for k, v in dependencies.items()},
}

with open(Path('./outputs/verification_report.json'), 'w') as f:
    json.dump(report, f, indent=2)

print(f"\\n📄 Reporte guardado en outputs/verification_report.json")

## 8. Resumen y Conclusiones

### ✅ Componentes Creados

1. **Aplicación Streamlit** (`app/demo.py`)
   - 6 métodos de detección (baseline, MC-Dropout, varianza decoder)
   - Calibración con/sin Temperature Scaling
   - Interfaz interactiva con controles
   - Visualización de incertidumbre

2. **Imágenes de Muestra** (`app/samples/`)
   - Casos fáciles, medios y difíciles
   - Diversidad de escenarios ADAS
   - Metadata de selección

3. **Documentación** (`README.md`)
   - Instrucciones de ejecución
   - Interpretación de resultados
   - Casos de uso en ADAS

4. **Scripts de Lanzamiento**
   - PowerShell para Windows
   - Bash para Linux/Mac
   - Verificación de dependencias

5. **Capturas Comparativas** (`outputs/screenshots/`)
   - Antes/después calibración
   - Visualización de incertidumbre
   - Listas para defensa

### 🎯 Objetivos Cumplidos

✅ Demo muestra detección OVD en escenas ADAS  
✅ Compara confianza calibrada vs sin calibrar  
✅ Visualiza incertidumbre epistémica  
✅ Permite filtrado por umbral de incertidumbre  
✅ Conecta con métricas globales (Fase 5)  
✅ Interfaz intuitiva para audiencia no técnica  

### 📊 Casos de Uso Demostrados

1. **Calibración mejora honestidad**
   - Probabilidades ajustadas a precisión real
   - Reduce sobreconfianza del modelo

2. **Incertidumbre indica riesgo**
   - Alta incertidumbre → verificación necesaria
   - Baja incertidumbre → decisión automática

3. **Aplicación en ADAS**
   - Modo seguro: solo detecciones confiables
   - Alertas inteligentes según incertidumbre

### 🎓 Para la Defensa

La demo permite explicar visualmente:
- Qué es calibración y por qué importa
- Cómo se mide incertidumbre epistémica
- Por qué es crítico en sistemas de seguridad
- Trade-offs entre métodos (velocidad vs precisión)

### 📈 Próximos Pasos

1. Ejecutar demo y tomar capturas clave
2. Preparar escenarios para presentación:
   - Caso donde calibración corrige sobreconfianza
   - Caso donde incertidumbre detecta error
   - Comparación de velocidad entre métodos
3. Integrar en presentación de defensa
4. Documentar en tesis (capítulo de resultados)

---

## 🚀 INSTRUCCIONES DE EJECUCIÓN

### Opción 1: Ejecutar desde este notebook

```python
# Ejecutar todas las celdas en orden (1-7)
# Esto creará todos los archivos necesarios
```

### Opción 2: Ejecutar demo directamente

#### En Windows (PowerShell):
```powershell
cd "fase 6"
.\launch_demo.ps1
```

#### En Linux/Mac:
```bash
cd "fase 6"
./launch_demo.sh
```

#### Manual:
```bash
cd "fase 6"
streamlit run app/demo.py
```

### 📝 Notas Importantes

1. **Primera ejecución**: Ejecutar todas las celdas del notebook para crear archivos
2. **Ejecuciones siguientes**: Usar scripts de lanzamiento directamente
3. **Imágenes de muestra**: La celda 3 copia imágenes de BDD100K
4. **Dependencias**: Fase 4 (temperaturas) y Fase 5 (métricas) deben estar completas
5. **Performance**: MC-Dropout es ~5x más lento que baseline

### 🔍 Troubleshooting

- **Error "app/demo.py not found"**: Ejecutar celda 2
- **No se ven imágenes**: Ejecutar celda 3
- **Streamlit no instalado**: Ejecutar celda 1
- **Modelo no carga**: Verificar rutas en /opt/program/GroundingDINO/

### 📸 Capturar Screenshots

1. Ejecutar demo
2. Seleccionar diferentes métodos
3. Usar imágenes de muestra (easy/medium/hard)
4. Capturar pantalla con comparaciones
5. Guardar en `outputs/screenshots/` para defensa